In [188]:
import os
import sys
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import treetaggerwrapper

In [194]:
#extracting filenames
path = '/home/yassine/EMSE 2015-2016/Projet Recherche/Author-Verification-/corpus-english-sample'
filenames = []
    
for root, dirs, files in os.walk(path):
    for directory in sorted(dirs):
        #print directory[-2:]  #corpus
        for r, d, f in os.walk(path+'/'+directory):
            for name in sorted(f):
                filenames.append(path+'/'+directory+'/'+name) #populating filenames array

In [170]:
#char 8-gram tfidf feature
tfidf = TfidfVectorizer(input='filename',use_idf=True,analyzer='char',ngram_range=(8,8))
eight_char = tfidf.fit_transform(filenames).toarray()
print eight_char

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.02253446  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.01986979 ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
10
16469


In [84]:
#punctiation frequence of (',' ':' ';' '(' ')' '!') in text (normaized by thenumber of sentences)
def punctuation(text):
    sentences = text.replace('\n','')
    sentences = sentences.replace('.','\n').splitlines()
    matrix = np.zeros((1, 6))
    matrix[0, 0] = text.count(',')/float(len(sentences))
    matrix[0, 1] = text.count(':')/float(len(sentences))
    matrix[0, 2] = text.count(';')/float(len(sentences))
    matrix[0, 3] = text.count('(')/float(len(sentences))
    matrix[0, 4] = text.count(')')/float(len(sentences))
    matrix[0, 5] = text.count('!')/float(len(sentences))
    return matrix

train = ""
i = 0
for doc in filenames:
    f = open(doc,"r")
    train = f.read()
    tmp = punctuation(train)
    if i == 0: 
            punct = tmp
            i += 1
    else: 
        punct = np.concatenate((punct,tmp))
print punct

[[ 0.8         0.          0.025       0.          0.          0.2       ]
 [ 0.86206897  0.          0.10344828  0.          0.          0.27586207]
 [ 2.          0.          0.23076923  0.07692308  0.07692308  0.84615385]
 [ 1.34615385  0.          0.03846154  0.          0.          0.30769231]
 [ 0.5         0.10714286  0.17857143  0.          0.          0.67857143]
 [ 0.66666667  0.          0.07692308  0.          0.          0.15384615]
 [ 1.04545455  0.09090909  0.          0.          0.          0.81818182]
 [ 1.05882353  0.35294118  0.11764706  0.          0.          0.52941176]
 [ 0.53521127  0.          0.          0.          0.          0.11267606]
 [ 1.03846154  0.          0.          0.          0.          0.        ]]


In [90]:
#words number of stop words in text / number of total words in text
8char = tfidf.fit_transform(filenames).toarray()
print 8char
def vocabulary(text):
    count = CountVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english')
    countTotal = CountVectorizer(analyzer='word',ngram_range=(1,1))
    counter = count.fit_transform([text]).toarray()
    countT = countTotal.fit_transform([text]).toarray()
    matrix = np.zeros((1, 1))
    matrix[0, 0] = (countT.sum()-counter.sum())/float(countT.sum())

    return matrix

train = ""
i = 0
for doc in filenames:
    f = open(doc,"r")
    train = f.read()
    tmp = vocabulary(train)
    if i == 0: 
            vocab = tmp
            i += 1
    else: 
        vocab = np.concatenate((vocab,tmp))
print vocab

[[ 0.4770318 ]
 [ 0.51456311]
 [ 0.50340136]
 [ 0.49693252]
 [ 0.50153846]
 [ 0.50416667]
 [ 0.57053292]
 [ 0.52805281]
 [ 0.41538462]
 [ 0.55033557]]


In [97]:
#Average and standard deviation of words per sentence
def phrases(text):
    sentences = text.replace('\n','')
    sentences = sentences.replace('.','\n').splitlines()
    wps = np.array([len(s.split()) for s in sentences]) #wps contains the number of words in each sentence
    matrix = np.zeros((1, 5))
    matrix[0, 0] = wps.std() #deviation
    matrix[0, 1] = wps.mean() #mean
    matrix[0, 2] = np.amin(wps) #min
    matrix[0, 3] = np.amax(wps) #max
    matrix[0, 4] = len(sentences) #number of sentences
    return matrix

train = ""
i = 0
for doc in filenames:
    f = open(doc,"r")
    train = f.read()
    tmp = phrases(train)
    if i == 0: 
            phrase = tmp
            i += 1
    else: 
        phrase = np.concatenate((phrase,tmp))
print phrase

[[   4.38748219    6.5           1.           18.           40.        ]
 [   7.66262452   10.20689655    2.           30.           29.        ]
 [  30.79863979   23.53846154    1.          109.           13.        ]
 [   7.71573356   12.92307692    4.           30.           26.        ]
 [   9.91159906   10.78571429    1.           37.           28.        ]
 [  13.36800226   12.25641026    1.           84.           39.        ]
 [   8.83655537   14.22727273    3.           30.           22.        ]
 [  11.48792054   18.29411765    3.           53.           17.        ]
 [   4.26102314    5.11267606    0.           24.           71.        ]
 [   7.5499324    10.80769231    1.           27.           26.        ]]


In [195]:
#convert text to tags using TreeTagger wrapper for Python
def text_to_tags(text):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en',TAGDIR='/home/yassine/EMSE 2015-2016/Projet Recherche/tree-tagger-linux-3.2')
    tags = treetaggerwrapper.make_tags(tagger.tag_text(unicode(text,encoding='utf-8')))
    pos_tags = []
    for pos in tags:
        pos_tags.append(pos[1])
    return " ".join(pos_tags)

#print text_to_tags('Hello world, My name is Yassine better, I had taken taking best better !') #example


#write documents as tags to file (RUNS ONCE!)
for root, dirs, files in os.walk(path):
    for directory in sorted(dirs):
        for r, d, f in os.walk(path+'/'+directory):
            for name in sorted(f):
                f = open(path+'/'+directory+'/'+name,"r")
                train = f.read()
                tags = text_to_tags(train)
                name = re.sub('\.txt$', '', name)
                f_tags = open(path+'/'+directory+'/'+name + '_tags.txt',"w")
                f_tags.write(tags)
                
    

In [184]:
#contructing pos frequency and 4-pos grams
def four_pos(text):
    tags = text_to_tags(text)
    countTotal = CountVectorizer(analyzer='word',ngram_range=(1,1),token_pattern=r"\b\w+\b")
    transformer = TfidfTransformer()
    countT = countTotal.fit_transform([tags]).toarray()
    mat_pos = transformer.fit_transform(countT).toarray()
    return mat_pos

train = ""
i = 0
tmp = []
for doc in filenames:
    f = open(doc,"r")
    train = f.read()
    print len(four_pos(train)[0])
    tmp.append(four_pos(train)[0])
#pos_grams = np.array(tmp)
#print len(pos_grams)




28
33
30
31
32
35
32
27
32
34
